###Домашнее задание к уроку #2

1.Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)<br>
2.Повторить п.1, но используя уже не медиану, а max<br>
3.*Повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)<br>
подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал<br>
подсказка 2: нужен именно idf, как вес.<br>
4.Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score.<br>
5.Сделать самостоятельные выводы и предположения о том, почему тот или иной способ оказался эффективнее остальных.

In [1]:
import pandas as pd

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt 

--2022-03-27 15:24:19--  https://docs.google.com/uc?export=download&confirm=t&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR
Resolving docs.google.com (docs.google.com)... 173.194.215.102, 173.194.215.113, 173.194.215.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.215.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/iir4rumueo54g3a09cfl2p3r30iojbeq/1648394625000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download [following]
--2022-03-27 15:24:19--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/iir4rumueo54g3a09cfl2p3r30iojbeq/1648394625000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-

In [3]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [4]:
news.iloc[0]['title']

'Заместитель председателяnправительства РФnСергейnИвановnизбран председателем советаnПБК ЦСКАn. Как сообщаетnпресс-службаnармейцев, такое решение было единогласно принято на первом заседании совета клуба. Основной функцией этого органа будет обсуждение текущего состояния, планирование и утверждение стратегии развития клуба на ближайшую перспективу.nnВ состав совета вошли:nГеоргий Полтавченкоn(полномочный представитель президента в Центральном федеральном округе),nАлексей Седовn(генерал-полковник, начальник службы по защите конституционного строя и борьбе с терроризмомnФСБ РФn),nАлександр Новакn(заместитель министра финансов РФ),nВячеслав Полтавцевn(заместитель генерального директора, руководитель блока по взаимодействию с органами власти, социальной и корпоративной политикиnГМК «Норильский никель»n),nЛеонид Исаковичn(генеральный директор ООО «УК «Спортивные проекты»),nОльга Плешаковаn(генеральный директорn«Трансаэро»n),nАлександр Провоторовn, (генеральный директорnОАО «Ростелеком»n), А

In [5]:
!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv

--2022-03-27 15:24:23--  https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a
Resolving drive.google.com (drive.google.com)... 173.194.215.100, 173.194.215.113, 173.194.215.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.215.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jc3tu18tg4o7rnpl9afabdlv5qc35fcm/1648394625000/14904333240138417226/*/1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a?e=download [following]
--2022-03-27 15:24:23--  https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jc3tu18tg4o7rnpl9afabdlv5qc35fcm/1648394625000/14904333240138417226/*/1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a?e=download
Resolving doc-04-c0-docs.googleusercontent.com (doc-04-c0-docs.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Connecting to doc-04-c0-docs.googleusercontent.com (doc-04-c0

Загрузим пользователей и списки последних прочитанных новостей

In [6]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### 1. Получаем векторные представления новостей

In [7]:
!pip install razdel pymorphy2 pyLDAvis

     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 1.7 MB 9.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 8.2 MB 29.5 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a3c382d5186c6916cc9d1354fd40895232cdd44dfefa2b62f75887461b4f0604
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [8]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [11]:
stopword_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [12]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2022-03-27 15:24:44--  https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c
Resolving drive.google.com (drive.google.com)... 173.194.215.138, 173.194.215.100, 173.194.215.113, ...
Connecting to drive.google.com (drive.google.com)|173.194.215.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i0u0lr77lqkdlmn1dafp4vdhvdtkh94v/1648394625000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download [following]
--2022-03-27 15:24:44--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i0u0lr77lqkdlmn1dafp4vdhvdtkh94v/1648394625000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-0s-c0

In [13]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [14]:
stopword_ru[200:210]

['ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам']

In [15]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [16]:
news['title'].iloc[:10].apply(lambda x: clean_text(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Possible nested set at position 39
  


0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [17]:
%%time
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

100%|██████████| 27000/27000 [00:35<00:00, 764.85it/s] 


CPU times: user 29.7 s, sys: 1.77 s, total: 31.4 s
Wall time: 35.4 s


In [18]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [19]:
news['title'].iloc[:2].apply(lambda x: lemmatization(x))

0    [заместитель, председатель, правительство, рф,...
1    [матч, финал, кубок, россия, футбол, приостано...
Name: title, dtype: object

In [20]:
%%time
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 27000/27000 [04:30<00:00, 99.78it/s] 


CPU times: user 4min 26s, sys: 2.07 s, total: 4min 28s
Wall time: 4min 30s


А теперь в 3 строчки обучим нашу модель

In [21]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [28]:
N_topic = 30 #можно менять

In [29]:
%%time
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary)#, passes=10)

CPU times: user 55.2 s, sys: 26.9 s, total: 1min 22s
Wall time: 53.3 s


In [30]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [31]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [32]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(1, 0.69815725), (2, 0.031251896), (20, 0.02927843), (25, 0.21964581)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [33]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: самолёт сша корабль полёт территория российский часть
topic_1: рак статья рейс спрос автор публикация первый
topic_2: смерть квартира возраст больной мужчина жизнь найти
topic_3: земля космический спутник тело запуск орбита поверхность
topic_4: военный американский россия действие советский боевой война
topic_5: украина украинский газпром киев белоруссия крым россия
topic_6: статья убийство километр египет сотрудник топливо px
topic_7: взрыв белый вирус фсб блок ук предполагать
topic_8: дом район произойти пострадать место здание пожар
topic_9: рубль фонд размер срок уголовный млн преступление
topic_10: ракета товар испытание дональд физика двигатель новый
topic_11: операция мозг пенсия министерство врач продукция чиновник
topic_12: путин луна россия век следствие владимир венесуэла
topic_13: всё государство эксперт банк российский социальный поток
topic_14: млн место рейтинг составить кремль болезнь сенатор
topic_15: журнал всё планета жизнь очень большой первый
topic_16: тем

Большинство тем вполне можно описать.

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [34]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [35]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.01437111, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.68454313, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.26387328, 0.        , 0.02961016,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [39]:
%%time
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]

CPU times: user 43.7 s, sys: 25.1 s, total: 1min 8s
Wall time: 41.7 s


In [40]:
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
0,6,0.0,0.014373,0.000000,0.0,0.0,0.0,0.0,0.684557,0.0,...,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0
1,4896,0.0,0.248986,0.000000,0.0,0.0,0.0,0.0,0.583517,0.0,...,0.000000,0.046282,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0
2,4897,0.0,0.698707,0.031278,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.029288,0.000000,0.0,0.000,0.0,0.219060,0.0,0.0,0.0,0.0
3,4898,0.0,0.191307,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.044884,0.491636,0.0,0.000,0.0,0.261603,0.0,0.0,0.0,0.0
4,4899,0.0,0.000000,0.047114,0.0,0.0,0.0,0.0,0.259220,0.0,...,0.000000,0.000000,0.0,0.195,0.0,0.000000,0.0,0.0,0.0,0.0


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

### 2. Получаем векторные представления пользователей

In [ ]:
users.head(3)

In [42]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [ ]:
doc_dict[293672]

In [65]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [66]:
# def get_user_embedding(user_articles_list, doc_dict):
#     user_articles_list = eval(user_articles_list)
#     user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
#     # print(user_vector)
#     user_vector = np.mean(user_vector, 0)  # среднее
#     return user_vector

In [67]:
# def get_user_embedding(user_articles_list, doc_dict):
#     user_articles_list = eval(user_articles_list)
#     user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
#     # print(user_vector)
#     user_vector = np.median(user_vector, 0)  # медиана
#     return user_vector

In [68]:
# def get_user_embedding(user_articles_list, doc_dict):
#     user_articles_list = eval(user_articles_list)
#     user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
#     # print(user_vector)
#     user_vector = np.max(user_vector, 0)  # макс
#     return user_vector

In [70]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.06117925, 0.        , 0.        , 0.02164276, 0.0963852 ,
       0.01009455, 0.0563345 , 0.00689891, 0.        , 0.04106871,
       0.00376387, 0.03768659, 0.11626814, 0.14462661, 0.00345674,
       0.01945305, 0.        , 0.09892312, 0.12888508, 0.00273857,
       0.03359338, 0.03023986, 0.04544017, 0.01477804, 0.        ,
       0.01663163, 0.        , 0.        , 0.        , 0.        ])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [72]:
FUNC = np.mean

In [75]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
0,u105138,0.014359,0.000000,0.002942,0.003883,0.025018,0.086401,0.022147,0.025035,0.021685,...,0.076338,0.008575,0.085434,0.030542,0.034824,0.029130,0.028455,0.00000,0.093721,0.007131
1,u108690,0.044946,0.000000,0.015165,0.000000,0.009274,0.126648,0.073111,0.010616,0.019652,...,0.036226,0.011855,0.015516,0.091913,0.009183,0.010581,0.009571,0.01408,0.009187,0.000000
2,u108339,0.041977,0.007303,0.006461,0.017232,0.027284,0.013430,0.033127,0.031632,0.109100,...,0.013050,0.014182,0.017397,0.047499,0.007179,0.011216,0.009412,0.00000,0.000000,0.002141


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [76]:
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

--2022-03-27 16:13:50--  https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK
Resolving drive.google.com (drive.google.com)... 172.217.203.101, 172.217.203.139, 172.217.203.102, ...
Connecting to drive.google.com (drive.google.com)|172.217.203.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4lc5g8h12b30r0gf2k83g0mji47dnhva/1648397625000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download [following]
--2022-03-27 16:13:50--  https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4lc5g8h12b30r0gf2k83g0mji47dnhva/1648397625000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download
Resolving doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Connecting to doc-0k-c0-docs.googleusercontent.com (doc-0k-c0

In [77]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [78]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29,churn
0,u105138,0.014359,0.000000,0.002942,0.003883,0.025018,0.086401,0.022147,0.025035,0.021685,...,0.008575,0.085434,0.030542,0.034824,0.029130,0.028455,0.00000,0.093721,0.007131,0
1,u108690,0.044946,0.000000,0.015165,0.000000,0.009274,0.126648,0.073111,0.010616,0.019652,...,0.011855,0.015516,0.091913,0.009183,0.010581,0.009571,0.01408,0.009187,0.000000,1
2,u108339,0.041977,0.007303,0.006461,0.017232,0.027284,0.013430,0.033127,0.031632,0.109100,...,0.014182,0.017397,0.047499,0.007179,0.011216,0.009412,0.00000,0.000000,0.002141,1


In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [80]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [83]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [84]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.20451687, 0.02761762, 0.31573161, 0.23342047, 0.0807191 ,
       0.04321743, 0.24789232, 0.12863708, 0.05650509, 0.35024801])

In [85]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


### Рассчитаем Precision, Recall, F_score

In [86]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [87]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}') 

Best Threshold=0.24323106054125598, F-Score=0.635, Precision=0.589, Recall=0.690


In [88]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9382824582824584

In [89]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.243231,0.635338,0.58885,0.689796,0.938282


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [90]:
FUNC = np.median

In [91]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
0,u105138,0.00000,0.0,0.0,0.00000,0.0,0.049861,0.000000,0.000000,0.000000,...,0.050222,0.000000,0.006425,0.000000,0.013851,0.0,0.0,0.0,0.082401,0.0
1,u108690,0.00000,0.0,0.0,0.00000,0.0,0.144419,0.076324,0.000000,0.000000,...,0.010063,0.000000,0.000000,0.101126,0.005263,0.0,0.0,0.0,0.000000,0.0
2,u108339,0.04141,0.0,0.0,0.00664,0.0,0.000000,0.021618,0.029915,0.097001,...,0.000000,0.016178,0.000000,0.041436,0.000000,0.0,0.0,0.0,0.000000,0.0


In [92]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29,churn
0,u105138,0.00000,0.0,0.0,0.00000,0.0,0.049861,0.000000,0.000000,0.000000,...,0.000000,0.006425,0.000000,0.013851,0.0,0.0,0.0,0.082401,0.0,0
1,u108690,0.00000,0.0,0.0,0.00000,0.0,0.144419,0.076324,0.000000,0.000000,...,0.000000,0.000000,0.101126,0.005263,0.0,0.0,0.0,0.000000,0.0,1
2,u108339,0.04141,0.0,0.0,0.00664,0.0,0.000000,0.021618,0.029915,0.097001,...,0.016178,0.000000,0.041436,0.000000,0.0,0.0,0.0,0.000000,0.0,1


In [93]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [94]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [95]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.13584276, 0.03265023, 0.39396846, 0.24151812, 0.05534674,
       0.06361513, 0.1625678 , 0.07683472, 0.06052051, 0.21765329])

In [96]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')   

Best Threshold=0.2235735676921352, F-Score=0.745, Precision=0.666, Recall=0.845


In [97]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9668562125704984

In [98]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.243231,0.635338,0.588850,0.689796,0.938282
1,median,0.223574,0.744604,0.665595,0.844898,0.966856


3. Повторить п.2, но используя уже не медиану, а max

In [99]:
FUNC = np.max

In [100]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
0,u105138,0.086156,0.000000,0.017654,0.023301,0.150111,0.316675,0.132880,0.150208,0.115487,...,0.236427,0.051449,0.392120,0.141799,0.151414,0.097702,0.096163,0.000000,0.266187,0.024889
1,u108690,0.237760,0.000000,0.057511,0.000000,0.040400,0.199144,0.143738,0.063696,0.092702,...,0.151137,0.038838,0.049067,0.176642,0.028789,0.049348,0.057429,0.084478,0.055121,0.000000
2,u108339,0.097085,0.025405,0.038769,0.048550,0.152275,0.047178,0.106711,0.091032,0.285114,...,0.059043,0.031431,0.077121,0.126276,0.027029,0.046107,0.028933,0.000000,0.000000,0.012847


In [101]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29,churn
0,u105138,0.086156,0.000000,0.017654,0.023301,0.150111,0.316675,0.132880,0.150208,0.115487,...,0.051449,0.392120,0.141799,0.151414,0.097702,0.096163,0.000000,0.266187,0.024889,0
1,u108690,0.237760,0.000000,0.057511,0.000000,0.040400,0.199144,0.143738,0.063696,0.092702,...,0.038838,0.049067,0.176642,0.028789,0.049348,0.057429,0.084478,0.055121,0.000000,1
2,u108339,0.097085,0.025405,0.038769,0.048550,0.152275,0.047178,0.106711,0.091032,0.285114,...,0.031431,0.077121,0.126276,0.027029,0.046107,0.028933,0.000000,0.000000,0.012847,1


In [102]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [103]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [104]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.08226717, 0.01184894, 0.8979433 , 0.26280353, 0.05585265,
       0.00574044, 0.45681185, 0.10666864, 0.04150975, 0.32425468])

In [105]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.3428098760469592, F-Score=0.761, Precision=0.726, Recall=0.800


In [106]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9679423222280364

In [107]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.243231,0.635338,0.588850,0.689796,0.938282
1,median,0.223574,0.744604,0.665595,0.844898,0.966856
2,amax,0.342810,0.761165,0.725926,0.800000,0.967942


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [109]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

# users['articles_str'] = users['articles'].apply(lambda x: " ".join([i.strip() for i in x.replace("[","").replace("]","").split(",")]), 

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [110]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [111]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [112]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.median(user_vector, axis=0)
    return user_vector

In [113]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|██████████| 8000/8000 [01:21<00:00, 98.03it/s]


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29
0,u105138,0.000000,0.0,0.0,0.000000,0.0,0.448624,0.000000,0.000000,0.000000,...,0.431964,0.000000,0.059713,0.000000,0.116038,0.0,0.0,0.0,0.699217,0.0
1,u108690,0.000000,0.0,0.0,0.000000,0.0,1.211176,0.643778,0.000000,0.000000,...,0.086555,0.000000,0.000000,0.861976,0.046783,0.0,0.0,0.0,0.000000,0.0
2,u108339,0.351097,0.0,0.0,0.057115,0.0,0.000000,0.185935,0.262587,0.798719,...,0.000000,0.142748,0.000000,0.341069,0.000000,0.0,0.0,0.0,0.000000,0.0


In [114]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29,churn
0,u105138,0.000000,0.0,0.0,0.000000,0.0,0.448624,0.000000,0.000000,0.000000,...,0.000000,0.059713,0.000000,0.116038,0.0,0.0,0.0,0.699217,0.0,0
1,u108690,0.000000,0.0,0.0,0.000000,0.0,1.211176,0.643778,0.000000,0.000000,...,0.000000,0.000000,0.861976,0.046783,0.0,0.0,0.0,0.000000,0.0,1
2,u108339,0.351097,0.0,0.0,0.057115,0.0,0.000000,0.185935,0.262587,0.798719,...,0.142748,0.000000,0.341069,0.000000,0.0,0.0,0.0,0.000000,0.0,1


In [115]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [116]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [117]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([1.36744111e-02, 3.04775896e-05, 9.84656998e-01, 2.85432836e-01,
       4.06969431e-03, 1.00502237e-03, 2.59226362e-02, 1.30542116e-02,
       5.99821286e-03, 1.08364810e-01])

In [118]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')  

Best Threshold=0.44110532965773797, F-Score=0.889, Precision=0.877, Recall=0.902


In [119]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9903901389615676

In [120]:
metrics_df = metrics_df.append({
    'model': 'idf_mean',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [121]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.243231,0.635338,0.588850,0.689796,0.938282
1,median,0.223574,0.744604,0.665595,0.844898,0.966856
2,amax,0.342810,0.761165,0.725926,0.800000,0.967942
3,idf_mean,0.441105,0.889336,0.876984,0.902041,0.990390
